In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
filtered_df = pd.read_csv('results/filtered_netflix_reviews_language.csv')

In [3]:
non_eng_df = filtered_df[filtered_df['language']!='en']
eng_df = filtered_df[filtered_df['language']=='en']

In [4]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)  # Move model to GPU
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)

translated_content = []

for i in range(len(non_eng_df)):
    text = non_eng_df['content'].iloc[i]
    tokenizer.src_lang = non_eng_df['language'].iloc[i] + "_XX"
    
    # Tokenize and move tensors to GPU
    encoded_hi = tokenizer(text, return_tensors="pt").to(device)
    
    # Generate tokens and decode
    generated_tokens = model.generate(
        **encoded_hi, 
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
    )
    out = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    translated_content.append(out[0])
    print('not english', i)

# Add translations to the DataFrame
non_eng_df['translated_content'] = translated_content


c:\Users\Babli Dey\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


not english 0
not english 1
not english 2
not english 3
not english 4
not english 5
not english 6
not english 7
not english 8
not english 9
not english 10
not english 11
not english 12
not english 13
not english 14
not english 15
not english 16
not english 17
not english 18
not english 19
not english 20
not english 21
not english 22
not english 23
not english 24
not english 25
not english 26
not english 27
not english 28
not english 29
not english 30
not english 31
not english 32
not english 33
not english 34
not english 35
not english 36
not english 37
not english 38
not english 39
not english 40
not english 41
not english 42
not english 43
not english 44
not english 45
not english 46
not english 47
not english 48
not english 49
not english 50
not english 51
not english 52
not english 53
not english 54
not english 55
not english 56
not english 57
not english 58
not english 59
not english 60
not english 61
not english 62
not english 63
not english 64
not english 65
not english 66
not e

C:\Users\Babli Dey\AppData\Local\Temp\ipykernel_8592\245120131.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_eng_df['translated_content'] = translated_content


In [7]:
non_eng_df.to_csv('results/non_eng_translated.csv', index=False)

In [8]:
non_english_hel_df = filtered_df[filtered_df['language']!='en']

In [9]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langdetect import detect

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# Function to translate text to English
def translate_to_english(text):
    try:
        # Tokenize the input text
        inputs = tokenizer.encode(text, return_tensors="pt", truncation=True).to(device)
        
        # Generate translation
        outputs = model.generate(inputs, max_length=512, num_beams=5, early_stopping=True)
        
        # Decode the output
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated_text
    except Exception as e:
        return f"Translation failed: {e}"


# Add detected language and translation columns
non_english_hel_df["language_hel"] = non_english_hel_df["content"].apply(detect_language)
non_english_hel_df["translated"] = non_english_hel_df["content"].apply(translate_to_english)

# Display the DataFrame
print(non_english_hel_df)


c:\Users\Babli Dey\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Babli Dey\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


c:\Users\Babli Dey\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


C:\Users\Babli Dey\AppData\Local\Temp\ipykernel_8592\3250758195.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_english_hel_df["language_hel"] = non_english_hel_df["content"].apply(detect_language)


                                    reviewId                userName  \
1       de92d64e-999b-4cbd-8487-518126fe0629  goat and birds channel   
3       258242e4-ba13-4233-88a0-147c721fd65f           Shubham Kadam   
5       43794861-0f4a-4e5e-95e5-197c1ba643cb          Basudev Hansda   
11      dff952e1-590e-4382-84bd-0193210192ac        Laras Prameswari   
16      7adc5e67-55b9-47e6-803e-f27fad312f11           Ayesha cheema   
...                                      ...                     ...   
123347  8caddae9-07eb-41fd-a19f-cfaa44f8da45          Alison Stevens   
123355  a19d21f6-8191-40d1-8673-3d2295a4be2b       Reddy reddy Reddy   
123365  6d64b64b-2964-479b-a94e-e75150434ee8                   Aaron   
123406  919526ef-ec18-446b-9a3a-de8985c171e6                Ali Rezk   
123420  0495bfb1-c7d2-4890-9b54-9ce95e19fda9              Karan Sahu   

                                                  content  score  \
1                                                NICE APP      5   

C:\Users\Babli Dey\AppData\Local\Temp\ipykernel_8592\3250758195.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_english_hel_df["translated"] = non_english_hel_df["content"].apply(translate_to_english)


In [12]:
non_english_hel_df.to_csv('results/non_eng_translated_hel.csv', index=False) 